In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo
import mysql.connector as SQLC
import mysql.connector

ModuleNotFoundError: No module named 'mysql'

### Testes:

In [ ]:
import mysql.connector as SQLC

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)

Cursor = DataBase.cursor()

Cursor.execute("CREATE TABLE genbank (cds varchar(300)")
print("Tabela criada")  

In [ ]:
database = 'nucleotide'
word = 'GLI2 '
res= input()
email= 'rodrigoce9@gmail.com'
Entrez.email= email
handle_search=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle_search)
handle_search.close()
idlist= record['IdList']

#Ids= []
#Pubmed=[]
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
for info in records:
    a= info.id
    Ids.append(a)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            featcds.append(i)  #this is appending the index of where CDS is, not the quantity of the CDS
            CDS=(str(i))
            A= len(CDS)
            featcds.append(A)
            print (len(CDS))

In [ ]:
import mysql.connector as SQLC
import mysql.connector

database = 'nucleotide'
word = 'GLI2 '
res= input()
email= 'rodrigoce9@gmail.com'
Entrez.email= email
handle_search=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle_search)
handle_search.close()
idlist= record['IdList']

Ids= []
#Pubmed=[]
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
for info in records:
    a= info.id
    Ids.append(a)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            featcds.append(i)
            CDS=(str(i))
            A= len(CDS)
            B= str(A)
#             print (len(CDS))
print (featcds)
tab = ("INSERT INTO genbank (cds) VALUES ('{}')".format(value)) # %s overwrites the same value
    
# val = (featcds)
# tab = ('''INSERT INTO genbank (cds) VALUES (%s)''', (val))
#sql= tab + val


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for value in val:#instead of using loop we can use Cursor.executemany(tab, val)
        Cursor.execute(tab)
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [ ]:
for value in val:#instead of using loop we can use Cursor.executemany(tab, val)
    tab = ("INSERT INTO genbank (cds) VALUES ('{}')".format(value)) # %s overwrites the same value
    

### Start povoation:

### Extração de Ids:

In [4]:
import time
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    
import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
print(n_genes)



escolha o que quer pesquisar: irs
escolha o nº de genes que quer obter (máximo 20): 10
['8223391', '64880', '3376', '45785', '408438', '111506503', '39258', '125265521', '110376740', '25467']


In [8]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    #print(info.id)
print (Ids)

['AL329769.1', 'X61083.1', 'U02356.1', 'AASW01042081.1', 'Y00554.1', 'EL279606.1', 'EC941813.1']


In [7]:
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    #print(info.id)
    #print(info.description)
    description.append(info.description)
    print (description)

['Tetraodon nigroviridis genome survey sequence T3 end of clone 012H02 of library C from Tetraodon nigroviridis, genomic survey sequence']
['Tetraodon nigroviridis genome survey sequence T3 end of clone 012H02 of library C from Tetraodon nigroviridis, genomic survey sequence', 'R.meliloti plasmid derived nodL gene for prabable acetyltransferase']
['Tetraodon nigroviridis genome survey sequence T3 end of clone 012H02 of library C from Tetraodon nigroviridis, genomic survey sequence', 'R.meliloti plasmid derived nodL gene for prabable acetyltransferase', 'Profundulus guatemalensis tyrosine kinase (X-src) gene, partial cds']
['Tetraodon nigroviridis genome survey sequence T3 end of clone 012H02 of library C from Tetraodon nigroviridis, genomic survey sequence', 'R.meliloti plasmid derived nodL gene for prabable acetyltransferase', 'Profundulus guatemalensis tyrosine kinase (X-src) gene, partial cds', 'Drosophila simulans strain SIM6 Cont6415.2, whole genome shotgun sequence']
['Tetraodon 

In [6]:
import mysql.connector as SQLC
import mysql.connector

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try: 
    Cursor.execute("SET GLOBAL sql_mode = ''")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

ModuleNotFoundError: No module named 'mysql'

In [ ]:
import mysql.connector as SQLC
import mysql.connector

#tab = ("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value)) # %s overwrites the same value
val = (n_genes)
    
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for value in val:
        Cursor.execute("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value))
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    

In [2]:
import mysql.connector as SQLC
import mysql.connector

#tab = ("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value)) # %s overwrites the same value
des = (description)

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for value in n_genes:
        Cursor.execute(f"INSERT INTO Gene (ID_genebank)  VALUES ('{value}')")
        for values in des:
            Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
            print (values)
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

ModuleNotFoundError: No module named 'mysql'

In [ ]:
import mysql.connector
import pandas as pd 
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
####### end of connection ####
tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",my_conn)